# Data Cleaning experiment

In [2]:
from datacleaner import autoclean
import pandas as pd

In [3]:
path = '/Users/reneehall/Documents/Learning/lighthouse_labs/coursework-lighthouse-labs/Mid-term project/colab_data/'

fl_df = pd.read_csv(path+'balanced_sample.csv')

In [6]:
fl_df.isnull().sum().sort_values(ascending=False)

longest_add_gtime      0
wheels_on              0
taxi_out               0
dep_delay              0
dep_time               0
crs_dep_time           0
dest_city_name         0
dest                   0
dest_airport_id        0
origin_city_name       0
origin                 0
origin_airport_id      0
op_carrier_fl_num      0
tail_num               0
op_unique_carrier      0
mkt_carrier_fl_num     0
mkt_carrier            0
branded_code_share     0
mkt_unique_carrier     0
wheels_off             0
taxi_in                0
total_add_gtime        0
crs_arr_time           0
first_dep_time         0
late_aircraft_delay    0
security_delay         0
nas_delay              0
weather_delay          0
carrier_delay          0
distance               0
flights                0
air_time               0
actual_elapsed_time    0
crs_elapsed_time       0
dup                    0
diverted               0
cancellation_code      0
cancelled              0
arr_delay              0
arr_time               0


In [5]:
my_clean_data = autoclean(fl_df)

In [8]:
fl_df.to_csv('my_clean_data.csv', index=False)

In [9]:
fl_df.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime'],
      dtype='object')

In [10]:
cols=['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       
       'crs_arr_time',
       'crs_elapsed_time',
      'distance']

(600000, 41)

In [24]:
import pandas as pd
import numpy as np

# bring in data
path = '/Users/reneehall/Documents/Learning/lighthouse_labs/coursework-lighthouse-labs/Mid-term project/colab_data/'
flights = pd.read_csv(path+'mini_sample.csv')

my_clean_data = autoclean(fl_df)

# assign X and y
cols=[ 'op_unique_carrier',
       'origin_airport_id',
       'dest_airport_id', 
       'distance', 'origin_fl_density', 'dest_fl_density','month',
       'weekday',
       'timestamp', 'crs_arr_daytime', 'crs_dep_daytime',
       'origin_airport_size', 'dest_airport_size', 'season', 'dist_group',
       'dep_early_morning', 'mean_op_carrier_delay',
       'mean_delay_origin_airport', 'mean_delay_dest_airport',
       'mean_dep_delay_origin', 'mean_dep_delay_dest', 'predicted_speed',
       'dist_group_delay_ratio',
       'origin_percent_dep_delay','mean_mo_delay_origin_airport', 
       ]


# 'mkt_carrier_fl_num', 'op_carrier_fl_num', 'mkt_unique_carrier',

flights['arr_delay_flag'] = flights['arr_delay'].map(lambda x: 1 if x > 0 else 0)

X = flights[cols]
y = flights['arr_delay_flag'].astype('category')

In [25]:
X[['crs_arr_daytime', 'crs_dep_daytime','dep_early_morning']] = X[['crs_arr_daytime', 'crs_dep_daytime', 'dep_early_morning']].astype('category')

X = pd.get_dummies(X, columns=[
       'op_unique_carrier',
       'origin_airport_id',
       'dest_airport_id', 
       'month',
       'weekday',
       'origin_airport_size', 'dest_airport_size', 'season', 'dist_group',
       ])

In [26]:
# scale data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
Xsc = scaler.fit_transform(X)

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xsc, y, train_size=0.7, test_size=0.3, random_state=101)

In [28]:
#Model
import xgboost as xgb

model = xgb.XGBRegressor(objective='reg:squarederror',
        n_estimators=500,
      learning_rate=0.001,
     max_depth=10)

model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.001, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [29]:
y_pred=model.predict(X_test)


import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

# evaluate using MAE
MAE = mean_absolute_error(y_test, y_pred)
print(f"MAE: {MAE}")

# evaluate using R2
R2 = r2_score(y_test, y_pred)
print(f"R2: {R2}")

# evaluate with adjusted R2
adj_R2 = 1 - ((1 - R2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print(f"adj_R2: {adj_R2}")

# calculate MAPE
MAPE = mean_absolute_percentage_error(y_test,y_pred)
print(f"MAPE: {MAPE}")

MAE: 0.49835892447630564
R2: 0.00344971583588638
adj_R2: -0.054108442325637496
MAPE: 1122200235426674.5
